**Note**: This notebook uses `await` syntax. Run it in a Jupyter environment with IPython kernel, or use `asyncio.run()` wrapper for regular Python scripts.

# Multimodal Content with v-router 🖼️

This notebook demonstrates how to use v-router with multimodal content (images, PDFs, and Word documents). The v-router library provides a unified interface for sending images and documents to different LLM providers.

## Overview

v-router supports:
- **Images**: JPEG, PNG, GIF, WebP formats
- **Documents**: PDF files and Word documents (.docx)
- **Automatic conversion**: File paths are automatically converted to base64
- **Unified interface**: Same API across all providers

## Setup

First, let's set up our environment and imports:

## Quick Start: Using Local Files

The easiest way to send images and PDFs is by passing file paths directly:

In [1]:
# Example using the PDF file included in this repository
from v_router import Client, LLM, HumanMessage


# Create a client
client = Client(
    llm_config=LLM(
        model_name="claude-3-5-sonnet-20241022",
        provider="anthropic"
    )
)

# Send a PDF file by just passing its path
pdf_path = "providers/assets/gameboy_color.pdf"
messages = [
    HumanMessage(content=pdf_path),
    HumanMessage(content="What is this document about? Give me a brief summary."),
]

response = await client.messages.create(messages=messages, max_tokens=200)
print("PDF Summary:")
print(response.content)

2025-06-13 15:56:51,730 - v_router.router - INFO - Trying primary model: claude-3-5-sonnet-20241022 on anthropic


PDF Summary:
["This is an instruction manual/booklet for the Nintendo Game Boy Color handheld video game system. The document covers various aspects including:\n\n1. Introduction to the system and its features\n2. Components and parts identification\n3. Basic setup and operation instructions\n4. Battery installation and maintenance\n5. Information about compatible Game Paks\n6. Two-player gaming setup using Game Link Cable\n7. Troubleshooting guide\n8. Warranty information\n9. Parts list and order form\n\nThe manual provides detailed technical specifications, safety warnings, and instructions for proper use of the Game Boy Color, which was a color screen upgrade to Nintendo's original Game Boy system. It includes diagrams and illustrations to help users understand the device's features and proper operation procedures.\n\nThe document appears to be an official Nintendo manual from when the Game Boy Color was released, containing all the necessary information for users to properly set up

## Sending Images

### Method 1: Base64 Encoded Images

You can send images by providing base64-encoded data directly:

In [2]:
import base64
import httpx
from pathlib import Path

# Import content types if you need to create multimodal messages manually
from v_router.classes.messages import TextContent, ImageContent, DocumentContent

In [4]:
# Download a sample image
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a7/Camponotus_flavomarginatus_ant.jpg/320px-Camponotus_flavomarginatus_ant.jpg"
response = httpx.get(image_url)
image_data = base64.b64encode(response.content).decode("utf-8")

# Create a client with Anthropic
client = Client(
    llm_config=LLM(
        model_name="claude-3-5-sonnet-20241022",
        provider="anthropic"
    )
)

# Send multimodal message
messages = [
    HumanMessage(content=[
        TextContent(text="What animal is in this image? Describe it in detail."),
        ImageContent(data=image_data, media_type="image/jpeg")
    ])
]

response = await client.messages.create(messages=messages, max_tokens=200)
print("Anthropic Response:")
print(response.content)

2025-06-13 15:57:19,812 - v_router.router - INFO - Trying primary model: claude-3-5-sonnet-20241022 on anthropic


Anthropic Response:
["This image shows a carpenter ant (genus Camponotus) in striking detail. The ant is captured in a dynamic pose, appearing to be rearing up on its hind legs, which is a common defensive posture. The ant's characteristic features are clearly visible, including its segmented body, long antennae, and powerful mandibles. The photo has a shallow depth of field, creating a beautifully blurred background that makes the ant stand out in sharp focus. The ant appears to be dark in color, possibly black or dark brown, and its exoskeleton has a slight sheen to it. Carpenter ants are among the larger ant species, and this image really showcases their impressive size and structure. The lighting in the photo gives it a warm, natural tone and helps highlight the ant's distinctive anatomy."]


### Method 2: File Path (Automatic Conversion)

v-router can automatically convert local image files to base64:

In [5]:
# Save the image locally
image_path = Path("/tmp/test_ant.jpg")
# Use the httpx response from earlier, not the v-router response
with open(image_path, "wb") as f:
    f.write(httpx.get(image_url).content)

# Send using file path - v-router will automatically convert to base64
messages = [
    HumanMessage(content=str(image_path))  # Just pass the file path as a string
]

response = await client.messages.create(messages=messages, max_tokens=100)
print("Response from file path:")
print(response.content)

# Clean up
image_path.unlink()

2025-06-13 15:57:32,364 - v_router.router - INFO - Trying primary model: claude-3-5-sonnet-20241022 on anthropic


Response from file path:
["This is a detailed macro photograph of an ant, showing its distinctive features like its segmented body, long antennae, and slender legs. The ant appears to be in a rearing or defensive posture, with its front legs raised off the ground. The image has a shallow depth of field, creating a soft, blurred background while keeping the ant in sharp focus. The lighting gives the photograph a warm, brownish tone, and the detail captured allows you to see the ant's"]


## Cross-Provider Compatibility

The same multimodal content works across different providers:

In [6]:
# Prepare the same multimodal message
multimodal_messages = [
    HumanMessage(content=[
        TextContent(text="What do you see in this image?"),
        ImageContent(data=image_data, media_type="image/jpeg")
    ])
]

# Test with different providers
providers = [
    ("anthropic", "claude-3-5-sonnet-20241022"),
    ("google", "gemini-1.5-flash"),
    ("openai", "gpt-4o")
]

for provider, model in providers:
    print(f"\n{provider.upper()} ({model}):")
    try:
        client = Client(
            llm_config=LLM(
                model_name=model,
                provider=provider
            )
        )
        response = await client.messages.create(messages=multimodal_messages, max_tokens=50)
        print(f"✓ {response.content}")
    except Exception as e:
        print(f"✗ Error: {e}")

2025-06-13 15:57:41,890 - v_router.router - INFO - Trying primary model: claude-3-5-sonnet-20241022 on anthropic



ANTHROPIC (claude-3-5-sonnet-20241022):


2025-06-13 15:57:44,749 - v_router.router - INFO - Trying primary model: gemini-1.5-flash on google


✓ ['This is a detailed macro photograph of an ant, showing its distinctive features like its segmented body, thin legs, and antennae. The ant appears to be in a rearing or alert posture, with its front legs raised off the surface']

GOOGLE (gemini-1.5-flash):


2025-06-13 15:57:46,045 - v_router.router - INFO - Trying primary model: gpt-4o on openai


✓ ["That's a close-up image of a carpenter ant (genus *Camponotus*) carrying something.  Specifically, it appears to be carrying another, smaller insect or possibly a piece of food.  The ant is black and its body is"]

OPENAI (gpt-4o):
✓ ['The image shows a close-up of an ant standing on a surface. The ant appears to be in a defensive or alert posture, with its body raised and legs spread. The background is blurred, focusing attention on the ant.']


## Word Documents (.docx)

v-router supports Word documents (.docx) by automatically converting them to HTML using the mammoth library. This works across all providers by sending the converted content as text.

### Method 1: File Path (Automatic Conversion)

The easiest way to send Word documents is by passing the file path directly:

In [7]:
# Example using the Word document included in this repository
from v_router import Client, LLM

# Create a client
client = Client(
    llm_config=LLM(
        model_name="claude-3-5-sonnet-20241022",
        provider="anthropic"
    )
)

# Send a Word document by just passing its path
docx_path = "providers/assets/order.docx"
messages = [
    HumanMessage(content=docx_path),
    HumanMessage(content="What is this document about? Give me a brief summary.")
]

response = await client.messages.create(messages=messages, max_tokens=200)
print("Word Document Summary:")
print(response.content)

2025-06-13 15:58:10,229 - v_router.router - INFO - Trying primary model: claude-3-5-sonnet-20241022 on anthropic


Word Document Summary:
['This is a Purchase Order (PO) document with PO number 23781 for automotive parts. The customer, John Smith from Redline Auto Center in Lansing, Michigan, is ordering the following items:\n\n1. 4 units of Brake Discs, Pads & Calipers at $111.36 each\n2. 2 units of Control Arm at $60.93 each\n3. 2 units of Suspension Lift Kit at $399.83 each\n\nThe subtotal is $1,366.96, with a 10% discount, 12% sales tax, additional costs for shipping & handling ($800) and other costs ($500), bringing the total amount to $2,694.30. The payment terms indicate that payment is due 30 days upon receipt of the items, and shipping terms are Freight on Board via Air & Land shipping method.']


### Method 2: Base64 Encoded Word Documents

You can also send Word documents by providing base64-encoded data directly:

In [8]:
# Load and encode a Word document
with open("providers/assets/order.docx", "rb") as f:
    docx_data = base64.b64encode(f.read()).decode("utf-8")

# Create a client with Anthropic
client = Client(
    llm_config=LLM(
        model_name="gpt-4.1-mini",
        provider="openai"
    )
)

# Send multimodal message with Word document
messages = [
    HumanMessage(content=[
        TextContent(text="Please analyze this Word document and tell me what type of document it is."),
        DocumentContent(
            data=docx_data, 
            media_type="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
        )
    ])
    ]

response = await client.messages.create(messages=messages, max_tokens=200)
print("Word Document Analysis:")
print(response.content)

2025-06-13 15:58:21,072 - v_router.router - INFO - Trying primary model: gpt-4.1-mini on openai


Word Document Analysis:
['Based on the content and structure of the provided document, this is a **Purchase Order**.\n\n### Key indicators:\n- The document title explicitly states **"PURCHASE ORDER"**.\n- It contains typical purchase order elements such as:\n  - **Company and vendor information** (names, addresses, contact details).\n  - **Purchase Order Number** and **Date**.\n  - Sections labeled **Vendor** and **Customer**.\n  - A detailed **itemized list** with columns for:\n    - Product Code\n    - Product Description\n    - Quantity\n    - Unit Price\n    - Amount\n  - Shipping terms and methods.\n  - Financial summary including:\n    - Subtotal\n    - Discount\n    - Sales Tax\n    - Other Costs\n    - Shipping & Handling\n    - Total Amount\n  - Payment terms note ("Payment shall be 30 days upon receipt of the items above.")\n\n### Conclusion:\nThis document is a **Purchase Order (PO)** used by']


### Cross-Provider Word Document Support

Word documents work across all providers through automatic HTML conversion:

In [9]:
# Prepare the same Word document message
word_doc_messages = [
    HumanMessage(content=[
        TextContent(text="What type of document is this?"),
        DocumentContent(
            data=docx_data, 
            media_type="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
        )
    ])
]

# Test with different providers
providers = [
    ("anthropic", "claude-3-5-sonnet-20241022"),
    ("google", "gemini-1.5-flash"),
    ("openai", "gpt-4o")
]

for provider, model in providers:
    print(f"\n{provider.upper()} ({model}):")
    try:
        client = Client(
            llm_config=LLM(
                model_name=model,
                provider=provider
            )
        )
        response = await client.messages.create(messages=word_doc_messages, max_tokens=50)
        print(f"✓ {response.content}")
    except Exception as e:
        print(f"✗ Error: {e}")

2025-06-13 15:58:30,062 - v_router.router - INFO - Trying primary model: claude-3-5-sonnet-20241022 on anthropic



ANTHROPIC (claude-3-5-sonnet-20241022):


2025-06-13 15:58:33,561 - v_router.router - INFO - Trying primary model: gemini-1.5-flash on google


✓ ['This is a Purchase Order (PO) document. This can be clearly seen in the header of the document where "PURCHASE ORDER" is prominently displayed, and it contains typical purchase order elements such as:\n\n1. PO Number (']

GOOGLE (gemini-1.5-flash):


2025-06-13 15:58:34,396 - v_router.router - INFO - Trying primary model: gpt-4o on openai


✓ ["This is a **Purchase Order (PO)**.  It's a commercial document issued by a buyer (Redline Auto Center in this case) to a seller (the company whose details are at the top) for the purchase of goods or services"]

OPENAI (gpt-4o):
✓ ['The document you provided is a "Purchase Order." A purchase order is a commercial document issued by a buyer to a seller, indicating the types, quantities, and agreed prices for products or services the seller will provide to the buyer. It serves as an']


### Complex Multimodal Messages with Word Documents

You can combine Word documents with images and other content types:

In [10]:
# Create a complex multimodal message with multiple content types
complex_messages = [
    HumanMessage(content=[
            TextContent(text="I'm sharing multiple documents with you:"),
            TextContent(text="1. A Word document:"),
            DocumentContent(
                data=docx_data, 
                media_type="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
            ),
            TextContent(text="2. An image:"),
            ImageContent(data=image_data, media_type="image/jpeg"),
            TextContent(text="Can you briefly describe what each contains?")
        ])
]

try:
    client = Client(
        llm_config=LLM(
            model_name="claude-3-5-sonnet-20241022",
            provider="anthropic"
        )
    )
    response = await client.messages.create(messages=complex_messages, max_tokens=300)
    print("Multi-Content Analysis:")
    print(response.content)
except Exception as e:
    print(f"Error with complex message: {e}")
    print("\nTrying with a simpler Word document message...")
    
    # Fallback to a simpler message
    simple_messages = [
        {
            "role": "user", 
            "content": [
                TextContent(text="What type of document is this?"),
                DocumentContent(
                    data=docx_data, 
                    media_type="application/vnd.openxmlformats-officedocument.wordprocessingml.document"
                )
            ]
        }
    ]
    response = await client.messages.create(messages=simple_messages, max_tokens=100)
    print(response.content)

2025-06-13 15:58:49,641 - v_router.router - INFO - Trying primary model: claude-3-5-sonnet-20241022 on anthropic


Multi-Content Analysis:
["Let me describe both documents:\n\n1. The Word document appears to be a purchase order template with the following key elements:\n- Header section for company details\n- PO Number: 23781\n- Vendor and customer information sections\n- Shipping terms: Freight on Board\n- Shipping method: Air & Land\n- Product list with 3 automotive parts (brake components, control arm, suspension kit)\n- Pricing calculations including subtotal, discount, tax, shipping and handling\n- Total amount: $2,694.30\n- Payment terms of 30 days\n\n2. The image shows a close-up macro photograph of an ant in a dramatic pose. The ant appears to be standing on a textured surface, and the photo captures fine details of the ant's body structure, including its antennae and legs. The background has a soft, blurred reddish tone while the ant is in sharp focus."]


## PDF Documents

Some providers (like Anthropic and Google) support PDF documents:

In [11]:
with open("providers/assets/gameboy_color.pdf", "rb") as f:
    pdf_data = base64.b64encode(f.read()).decode("utf-8")

# Send PDF to OpenAI
anthropic_client = Client(
    llm_config=LLM(
        model_name="gpt-4o",
        provider="openai"
    )
)

pdf_messages = [
    HumanMessage(content=[
        TextContent(text="Explain what is in this PDF"),
        DocumentContent(data=pdf_data, media_type="application/pdf")
    ])
]

response = await anthropic_client.messages.create(messages=pdf_messages, max_tokens=200)
print("PDF Analysis:")
print(response.content)

2025-06-13 15:58:57,435 - v_router.router - INFO - Trying primary model: gpt-4o on openai


PDF Analysis:
["The PDF is an instruction booklet for the Game Boy Color. Here's a summary of its contents:\n\n1. **Introduction**: Overview of the Game Boy Color video game system and its features.\n\n2. **List of Components**: Diagram and description of the Game Boy Color's parts, including the communication port, power indicator, and buttons.\n\n3. **Description of Components**: Detailed explanation of each component's function.\n\n4. **Installing Batteries**: Instructions on how to insert batteries into the device.\n\n5. **Caution**: Warning against using the Game Boy Rechargeable Battery Pack.\n\n6. **About Game Boy Color Game Paks**: Information on different types of game paks compatible with the system.\n\n7. **Using the Game Boy Color**: Steps to insert and play a game pak.\n\n8. **Changing the Screen Color**: Instructions on how to change screen colors for original Game Boy game paks.\n\n9. **Two-Player Mode Using the Game Link Cable**"]


## Complex Multimodal Conversations

You can combine multiple images and text in a single message:

In [12]:
# Use the ant image from earlier and download a new one
image1_data = image_data  # Reuse the ant image from earlier

# Download a different image
image2_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/320px-Cat03.jpg"
try:
    image2_response = httpx.get(image2_url)
    image2_data = base64.b64encode(image2_response.content).decode("utf-8")
except Exception as e:
    print(f"Failed to download second image: {e}")
    # Use a simple fallback example
    image2_data = image1_data

# Create a complex multimodal message
complex_messages = [
    HumanMessage(content=[
            TextContent(text="I'm going to show you two images."),
            TextContent(text="First image:"),
            ImageContent(data=image1_data, media_type="image/jpeg"),
            TextContent(text="Second image:"),
            ImageContent(data=image2_data, media_type="image/jpeg"),
            TextContent(text="Can you describe what you see in each image?")
        ])
]

try:
    response = await anthropic_client.messages.create(messages=complex_messages, max_tokens=300)
    print("Comparison Response:")
    print(response.content)
except Exception as e:
    print(f"Error with complex message: {e}")
    print("\nTrying with a simpler message...")
    
    # Fallback to a simpler message
    simple_messages = [
        {
            "role": "user", 
            "content": [
                TextContent(text="What do you see in this image?"),
                ImageContent(data=image1_data, media_type="image/jpeg")
            ]
        }
    ]
    response = await anthropic_client.messages.create(messages=simple_messages, max_tokens=100)
    print(response.content)

2025-06-13 15:59:23,017 - v_router.router - INFO - Trying primary model: gpt-4o on openai


Comparison Response:
['The first image shows an ant, likely a jumping ant, standing on a surface. It has a black body and is raising its front legs.\n\nThe second image features a ginger cat with orange fur and striking yellow eyes, looking directly at the camera. The background is slightly blurred.']


## Provider-Specific Considerations

### Anthropic
- Supports: Images (JPEG, PNG, GIF, WebP), PDFs, and Word documents (.docx)
- Max image size: 5MB per image
- Multiple images per message: Yes
- Word documents: Converted to HTML automatically

### Google (Gemini)
- Supports: Images, PDFs, and Word documents (.docx)
- Processes images through `inline_data`
- Multiple images per message: Yes
- Word documents: Converted to HTML automatically

### OpenAI
- Supports: Images and Word documents (.docx)
- Uses data URI format for images
- Multiple images per message: Yes
- Word documents: Converted to HTML automatically
- Note: PDFs will show a placeholder message

## Best Practices

1. **Image Optimization**: Resize large images before sending to reduce latency
2. **Error Handling**: Always handle provider-specific limitations
3. **Fallback Strategy**: Use v-router's fallback mechanism for providers that don't support certain content types
4. **Content Validation**: Ensure your content matches supported MIME types
5. **Word Document Format**: Use .docx format (not .doc) for best compatibility

## Example: Building a Visual Question Answering System

In [13]:
async def analyze_image(image_path: str, question: str, provider: str = "anthropic"):
    """Analyze an image and answer a question about it."""
    
    # Read and encode the image
    with open(image_path, "rb") as f:
        image_data = base64.b64encode(f.read()).decode("utf-8")
    
    # Determine MIME type
    import mimetypes
    mime_type, _ = mimetypes.guess_type(image_path)
    
    # Create client with fallback
    client = Client(
        llm_config=LLM(
            model_name="claude-3-5-sonnet-20241022" if provider == "anthropic" else "gpt-4o",
            provider=provider,
            try_other_providers=True  # Enable cross-provider fallback
        )
    )
    
    # Send the question with the image
    messages = [
        {
            "role": "user",
            "content": [
                TextContent(text=question),
                ImageContent(data=image_data, media_type=mime_type or "image/jpeg")
            ]
        }
    ]
    
    response = await client.messages.create(messages=messages, max_tokens=300)
    return response.content

# Example usage (you would need to provide an actual image path)
# result = await analyze_image("/path/to/image.jpg", "What objects can you identify in this image?")
# print(result)

## Summary

v-router makes it easy to work with multimodal content across different LLM providers:

- **Unified API**: Same interface for all providers
- **Automatic conversion**: File paths are converted to base64 automatically
- **Word document support**: .docx files are automatically converted to HTML using mammoth
- **Provider abstraction**: Handle provider differences transparently
- **Fallback support**: Automatically try other providers if one fails

This enables you to build robust multimodal applications without worrying about provider-specific implementation details.